In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
import tensorflow as tf
import numpy as np
import time
import h5py
import sys

In [ ]:
def fit(x, iters=1000, eps=1e-6):
    """
    Fits a 2-parameter Weibull distribution to the given data using maximum-likelihood estimation.
    :param x: 1d-ndarray of samples from an (unknown) distribution. Each value must satisfy x > 0.
    :param iters: Maximum number of iterations
    :param eps: Stopping criterion. Fit is stopped ff the change within two iterations is smaller than eps.
    :return: Tuple (Shape, Scale) which can be (NaN, NaN) if a fit is impossible.
        Impossible fits may be due to 0-values in x.
    """
    # fit k via MLE
    ln_x = np.log(x+eps)
    k = 1.
    k_t_1 = k

    for t in range(iters):
        x_k = x ** k
        x_k_ln_x = x_k * ln_x
        ff = np.sum(x_k_ln_x)
        fg = np.sum(x_k)
        f = ff / fg - np.mean(ln_x) - (1. / k)

        # Calculate second derivative d^2f/dk^2
        ff_prime = np.sum(x_k_ln_x * ln_x)
        fg_prime = ff
        f_prime = (ff_prime/fg - (ff/fg * fg_prime/fg)) + (1. / (k*k))

        # Newton-Raphson method k = k - f(k;x)/f'(k;x)
        k -= f/f_prime

        if np.isnan(f):
            return np.nan, np.nan
        if abs(k - k_t_1) < eps:
            break

        k_t_1 = k

    lam = np.mean(x ** k) ** (1.0 / k)

    return k, lam


def psi_i_dist(dist, k_i, lambda_i):
    """
    Gives the probability of sample inclusion
    :param dist: Numpy vector of distances between samples
    :param lambda_i: Scale of the Weibull fitting
    :param k_i: Shape of the Weibull fitting
    :return: PSI = Probability of Sample Inclusion. This is the probability that x' is included in the boundary estimated by x_i
    """
    return np.exp(-(((np.abs(dist))/lambda_i)**k_i))

def get_evm(data):
    D = pairwise_distances(X_evm_train, data, metric="euclidean", n_jobs=1)
    preds = np.zeros_like(D)
    for i in range(X_evm_train.shape[0]):
        for j in range(data.shape[0]):
            preds[i, j] = psi_i_dist(D[i, j], weibull_models[i][0], weibull_models[i][1])
    return preds

In [ ]:
tow = 50
weibull_models = []
for i in range(num_classes):
    train_ind = np.squeeze(np.where(y_train_n == i))
    D = pairwise_distances(X_evm_train[train_ind], X_evm_train, metric="euclidean", n_jobs=1)
    for j in range(D.shape[0]):
        D_tmp = np.sort(D[j])
        D_tmp = D_tmp[np.where(D_tmp>0)][:tow]
        weibull_models.append(fit(D_tmp, iters=100, eps=1e-6))
weibull_models = np.array(weibull_models)
weibull_models.shape

In [ ]:
evm = get_evm(X_evm_test)
np.mean(np.equal(y_train_n[np.argmax(evm, axis=0)], y_test_n))*100